In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline  
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from __future__ import division

//anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [44]:
train = pd.read_table("data_content", header=None, skip_blank_lines=False)
label = pd.read_table("label", header=None, dtype='category', skip_blank_lines=False)

In [45]:
train.columns = ['question']
label.columns = ['group']

In [46]:
print train.shape
print label.shape

(67213, 1)
(67213, 1)


In [48]:
ratio = 0.4
# Separate each muscle group data
chest = train[label['group']=='Chest']
chest_label = label[label['group']=='Chest']
chest_len = int(chest.shape[0]*ratio)

shoulder = train[label['group']=='Shoulders']
shoulder_len = int(shoulder.shape[0]*ratio)
shoulder_label = label[label['group']=='Shoulders']

back = train[label['group']=='Back']
back_label = label[label['group']=='Back']
back_len = int(back.shape[0]*ratio)

leg = train[label['group']=='Leg']
leg_label = label[label['group']=='Leg']
leg_len = int(leg.shape[0]*ratio)

tricep = train[label['group']=='Triceps']
tricep_label = label[label['group']=='Triceps']
tricep_len = int(tricep.shape[0]*ratio)

bicep = train[label['group']=='Biceps']
bicep_label = label[label['group']=='Biceps']
bicep_len = int(bicep.shape[0]*ratio)

ab = train[label['group']=='Abs']
ab_label = label[label['group']=='Abs']
ab_len = int(ab.shape[0]*ratio)

glute = train[label['group']=='Glutes']
glute_label = label[label['group']=='Glutes']
glute_len = int(glute.shape[0]*ratio)

In [49]:
# Take only the top 60% of each group of data
train_data = pd.concat([chest[0:chest_len], shoulder[0:shoulder_len], back[0:back_len], 
                        leg[0:leg_len], tricep[0:tricep_len], bicep[0:bicep_len], ab[0:ab_len], glute[0: glute_len]])
print train_data.shape
train_label = pd.concat([chest_label[0:chest_len], shoulder_label[0:shoulder_len], back_label[0:back_len], 
                   leg_label[0:leg_len], tricep_label[0:tricep_len], bicep_label[0:bicep_len], ab_label[0:ab_len],
                         glute_label[0: glute_len]])
print train_label.shape

(26883, 1)
(26883, 1)


In [22]:
label['group'].value_counts()

Back         16995
Leg          10390
Shoulders     8809
Chest         8685
Triceps       6864
Biceps        6798
Abs           6046
Glutes        2625
Name: group, dtype: int64

In [29]:
# separate data into training and test
X_train, X_test, y_train, y_test = train_test_split(data, train_label, test_size=0.2, random_state=0)
print X_train.shape
print X_test.shape

(32260, 8592218)
(8065, 8592218)


In [32]:
y_label = pd.Series(y_train['group'],dtype='category')
y_label.cat.categories 
y_label.cat.categories = [0,1,2,3,4,5,6,7]

In [33]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_label, test_size=0.2, random_state=0)
print X_train.shape
print y_train.shape
print X_valid.shape
print y_valid.shape

(25808, 8592218)
(25808,)
(6452, 8592218)
(6452,)


In [34]:
y_valid

[7, 1, 2, 1, 7, ..., 1, 6, 3, 5, 1]
Length: 6452
Categories (8, int64): [0, 1, 2, 3, 4, 5, 6, 7]

In [37]:
y_test = pd.Series(y_test['group'],dtype='category')
y_test.cat.categories = [0,1,2,3,4,5,6,7]

accuracy = float(np.sum(p_test == y_test) / y_test.shape[0])
print accuracy

0.233477991321
